In [1]:
import pandas as pd

file_path = 'ml-03-data-processing-songs-dataset.csv'

song_df = pd.read_csv('data/' + file_path)

In [2]:
# Drop the index column
song_df.drop(columns = ['Index'], inplace = True)
song_df.shape

(1994, 15)

In [3]:
# Drop nulls if you want to see them first.
#song_df['num_values'] = song_df.isnull().sum(axis = 1)

# Drop Nulls, better method
song_df.dropna(axis = 0, thresh = int(.5 * song_df.shape[1]), inplace = True)
song_df.dropna(axis = 1, thresh = int(.5 * song_df.shape[0]), inplace = True)

song_df.shape

(1987, 13)

In [4]:
song_df.dtypes.reset_index().set_index('index').T

index,Title,Artist,Top Genre,Year,Month,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Length (Duration),Acousticness,Speechiness,Popularity
0,object,object,object,float64,object,float64,float64,float64,float64,object,float64,float64,float64


In [5]:
# Match the month to a number

def match_month(month):
    if month.isnumeric():
        return int(month)
    try:
        return {
            'dec': 12,
            'nov': 11,
            'oct': 10,
            'sep': 9,
            'aug': 8,
            'jul': 7,
            'jun': 6,
            'may': 5,
            'apr': 4,
            'mar': 3,
            'feb': 2,
            'jan': 1
        }[month.lower()]
    except:
        return 'MATCH_FAIL' + month

song_df['Month'] = song_df['Month'].apply(match_month)

In [6]:
# Convert length column to float.
song_df['Length (Duration)'] = song_df['Length (Duration)'].apply(lambda x: str(x).replace(",","")).astype('float')

In [7]:
song_df.describe(include = 'all')

,Title,Artist,Top Genre,Year,Month,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Length (Duration),Acousticness,Speechiness,Popularity
count,1987,1984,1983,1984.000000,1987.000000,1983.000000,1983.000000,1983.000000,1983.000000,1982.000000,1984.000000,1986.00000,1985.000000
unique,1951,729,149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Hallelujah,Queen,album rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,3,37,410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,1959.468246,6.522899,120.234493,59.653555,-9.013616,19.019667,262.616549,28.916835,4.99144,59.511839
std,NaN,NaN,NaN,250.835292,3.419312,28.639923,22.170414,3.653139,16.750412,93.819708,29.028802,4.40705,14.366705
min,NaN,NaN,NaN,92.000000,1.000000,15.000000,3.000000,-27.000000,2.000000,93.000000,0.000000,2.00000,11.000000
25%,NaN,NaN,NaN,1978.000000,4.000000,99.000000,42.000000,-11.000000,9.000000,212.000000,3.000000,3.00000,49.000000
50%,NaN,NaN,NaN,1993.000000,7.000000,119.000000,61.000000,-8.000000,12.000000,245.000000,18.000000,4.00000,62.000000
75%,NaN,NaN,NaN,2007.000000,9.000000,136.000000,78.000000,-6.000000,23.000000,290.000000,50.000000,5.00000,71.000000


In [8]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1987 entries, 0 to 1993
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Title                   1987 non-null   object 
 1   Artist                  1984 non-null   object 
 2   Top Genre               1983 non-null   object 
 3   Year                    1984 non-null   float64
 4   Month                   1987 non-null   int64  
 5   Beats Per Minute (BPM)  1983 non-null   float64
 6   Energy                  1983 non-null   float64
 7   Loudness (dB)           1983 non-null   float64
 8   Liveness                1983 non-null   float64
 9   Length (Duration)       1982 non-null   float64
 10  Acousticness            1984 non-null   float64
 11  Speechiness             1986 non-null   float64
 12  Popularity              1985 non-null   float64
dtypes: float64(9), int64(1), object(3)
memory usage: 217.3+ KB


In [9]:
song_df[(song_df.Artist.isna())]

,Title,Artist,Top Genre,Year,Month,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Length (Duration),Acousticness,Speechiness,Popularity
886,Starman - 2012 Remaster,NaN,NaN,NaN,2,NaN,NaN,NaN,54.0,254.0,17.0,3.0,74.0
1109,Fire,NaN,dance pop,1978.0,6,NaN,NaN,NaN,NaN,NaN,20.0,3.0,56.0
1887,Respect,NaN,NaN,NaN,9,NaN,NaN,-5.0,5.0,NaN,16.0,4.0,73.0


In [10]:
outlier_range = song_df.describe(
    include = 'all'
).T.drop(columns = ['unique','count','top','freq','min','25%','50%','75%','max'])

outlier_range['outliers'] = (outlier_range['std'] * 3)


# Find and Display Outliers
no_outliers = []
for col in song_df.columns:
    if song_df[col].dtype == 'object':
        print(f'{col}                  Object - Skipping')
        no_outliers.append(col)
        continue
        
    outlier_string = f"Column: {col}\n"
    mean, std = song_df[col].mean(), song_df[col].std()
    outlier_found = 0
    for i in song_df.index.values:
        value = song_df[col][i]
        if (value > (mean + (std * 3)) or value < (mean - (std * 3))):
            outlier_string += f'Index: {i} | Value: {value} | Mean: {mean}\n'
            outlier_found += 1
    if outlier_found == 0:
        no_outliers.append(col)
        print(f'{col}                  Outliers Found: {outlier_found}')
    else:
        print(f'{col}                  Outliers Found: {outlier_found}')
            
    #print(outlier_string,"\n")
    


Title                  Object - Skipping
Artist                  Object - Skipping
Top Genre                  Object - Skipping
Year                  Outliers Found: 35
Month                  Outliers Found: 0
Beats Per Minute (BPM)                  Outliers Found: 4
Energy                  Outliers Found: 0
Loudness (dB)                  Outliers Found: 22
Liveness                  Outliers Found: 57
Length (Duration)                  Outliers Found: 26
Acousticness                  Outliers Found: 0
Speechiness                  Outliers Found: 42
Popularity                  Outliers Found: 9


In [11]:
song_df.columns


Index(['Title', 'Artist', 'Top Genre', 'Year', 'Month',
       'Beats Per Minute (BPM)', 'Energy', 'Loudness (dB)', 'Liveness',
       'Length (Duration)', 'Acousticness', 'Speechiness', 'Popularity'],
      dtype='object')

In [12]:
# I am interested in learning if the popularity can be predicted using all variables except for title and artist
song_df.drop(columns = ['Title','Artist'], inplace = True)

song_df.dtypes

Top Genre                  object
Year                      float64
Month                       int64
Beats Per Minute (BPM)    float64
Energy                    float64
Loudness (dB)             float64
Liveness                  float64
Length (Duration)         float64
Acousticness              float64
Speechiness               float64
Popularity                float64
dtype: object

In [13]:
song_df.shape

(1987, 11)

In [14]:
song_df = song_df[(song_df.Year != 92)]
song_df.shape

(1952, 11)

In [15]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1952 entries, 0 to 1993
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Top Genre               1948 non-null   object 
 1   Year                    1949 non-null   float64
 2   Month                   1952 non-null   int64  
 3   Beats Per Minute (BPM)  1948 non-null   float64
 4   Energy                  1948 non-null   float64
 5   Loudness (dB)           1948 non-null   float64
 6   Liveness                1948 non-null   float64
 7   Length (Duration)       1947 non-null   float64
 8   Acousticness            1949 non-null   float64
 9   Speechiness             1951 non-null   float64
 10  Popularity              1950 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 183.0+ KB


In [16]:
# IF I want to replace all the outliers with a value or nan

# import numpy as np

# num_cols = song_df.select_dtypes('number').columns
# df_subset = song_df.loc[:, num_cols]
# lim = np.abs((df_subset - df_subset.mean()) / df_subset.std(ddof=0)) < 3
# song_df.loc[:, num_cols] = song_df.where(lim, np.nan).head(50)

# song_df.dropna(axis = 0, thresh = (.5 * song_df.shape[1]), inplace = True)
# song_df

In [17]:
columns = song_df.columns.tolist()
columns

['Top Genre',
 'Year',
 'Month',
 'Beats Per Minute (BPM)',
 'Energy',
 'Loudness (dB)',
 'Liveness',
 'Length (Duration)',
 'Acousticness',
 'Speechiness',
 'Popularity']

In [18]:
from sklearn.impute import SimpleImputer
import numpy as np

def impute(df, column):
    
    if df[column].dtype == 'object':
        strat = 'most_frequent'
    else:
        strat = 'mean'
    
    imputer = SimpleImputer(missing_values=np.nan, strategy = strat)

    imputer.fit(df[[column]])
    SimpleImputer()

    X = df[[column]]
    return imputer.transform(X)

for good_col in columns:
    song_df[good_col] = impute(song_df, good_col)


In [19]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1952 entries, 0 to 1993
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Top Genre               1952 non-null   object 
 1   Year                    1952 non-null   float64
 2   Month                   1952 non-null   float64
 3   Beats Per Minute (BPM)  1952 non-null   float64
 4   Energy                  1952 non-null   float64
 5   Loudness (dB)           1952 non-null   float64
 6   Liveness                1952 non-null   float64
 7   Length (Duration)       1952 non-null   float64
 8   Acousticness            1952 non-null   float64
 9   Speechiness             1952 non-null   float64
 10  Popularity              1952 non-null   float64
dtypes: float64(10), object(1)
memory usage: 183.0+ KB


In [20]:
import requests
import re
from time import sleep

def check_year(artist, title):
    
    search_string = artist + " " + title + ' year'
    search_string = search_string.replace(" ", "+")
    search_path = 'https://www.google.com/search?q='
    try:
        sleep(1)
        page = requests.get(search_path + search_string).text

        years = re.findall('>[0-9]{4}<', page)
        found_year = years[0].replace('>',"").replace("<","")
        return found_year
    except:
        return None
# faulty_years = song_df.copy()
# faulty_years = faulty_years.loc[(song_df.Year == 92)]
# faulty_years.head()

#faulty_years['possible_year'] = faulty_years.apply(lambda row: check_year(row.Artist, row.Title), axis = 1)
#faulty_years[['Title','Artist','Year','possible_year']]
#faulty_years.possible_year.value_counts()

In [21]:
song_df.Year

0       2004.0
1       2000.0
2       2001.0
3       2007.0
4       2002.0
         ...  
1989    1958.0
1990    1958.0
1991    1959.0
1992    1959.0
1993    1959.0
Name: Year, Length: 1952, dtype: float64

In [23]:
song_df['Year'] = song_df['Year'].astype('int64').astype('str')
import datetime as dt

def get_age(date):
    delta = dt.datetime.today() - date
    return delta.days/365

song_df['Day'] = 1
song_df['date'] = pd.to_datetime(song_df[['Year','Month','Day']]) # can also use .assign(Day = 1)
song_df['age'] = song_df.date.apply(get_age)

In [24]:
song_df.drop(columns = ['Year','Month','Day','date'], inplace = True)


In [25]:
song_df

,Top Genre,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Length (Duration),Acousticness,Speechiness,Popularity,age
0,adult standards,157.0,30.0,-14.0,11.0,201.0,94.0,3.0,71.0,17.578082
1,album rock,135.0,79.0,-11.0,17.0,207.0,17.0,7.0,39.0,22.167123
2,alternative hip hop,168.0,69.0,-9.0,7.0,341.0,2.0,17.0,69.0,20.915068
3,alternative metal,173.0,96.0,-4.0,3.0,269.0,0.0,4.0,76.0,15.326027
4,classic rock,106.0,82.0,-5.0,10.0,256.0,1.0,3.0,59.0,19.747945
...,...,...,...,...,...,...,...,...,...,...
1989,adult standards,94.0,21.0,-12.0,11.0,128.0,84.0,7.0,63.0,64.112329
1990,adult standards,175.0,76.0,-8.0,76.0,136.0,73.0,6.0,69.0,63.945205
1991,blues rock,168.0,80.0,-9.0,31.0,162.0,74.0,7.0,74.0,63.112329
1992,bebop,174.0,26.0,-13.0,7.0,324.0,54.0,4.0,65.0,62.610959


In [32]:
prepped_for_ml = pd.get_dummies(song_df, 'Top Genre', drop_first = True)

In [42]:
prepped_for_ml

,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Length (Duration),Acousticness,Speechiness,Popularity,age,Top Genre_acoustic pop,...,Top Genre_reggae,Top Genre_reggae fusion,Top Genre_rock-and-roll,Top Genre_scottish singer-songwriter,Top Genre_soft rock,Top Genre_stomp and holler,Top Genre_streektaal,Top Genre_trance,Top Genre_uk pop,Top Genre_yacht rock
0,157.0,30.0,-14.0,11.0,201.0,94.0,3.0,71.0,17.578082,0,...,0,0,0,0,0,0,0,0,0,0
1,135.0,79.0,-11.0,17.0,207.0,17.0,7.0,39.0,22.167123,0,...,0,0,0,0,0,0,0,0,0,0
2,168.0,69.0,-9.0,7.0,341.0,2.0,17.0,69.0,20.915068,0,...,0,0,0,0,0,0,0,0,0,0
3,173.0,96.0,-4.0,3.0,269.0,0.0,4.0,76.0,15.326027,0,...,0,0,0,0,0,0,0,0,0,0
4,106.0,82.0,-5.0,10.0,256.0,1.0,3.0,59.0,19.747945,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,94.0,21.0,-12.0,11.0,128.0,84.0,7.0,63.0,64.112329,0,...,0,0,0,0,0,0,0,0,0,0
1990,175.0,76.0,-8.0,76.0,136.0,73.0,6.0,69.0,63.945205,0,...,0,0,0,0,0,0,0,0,0,0
1991,168.0,80.0,-9.0,31.0,162.0,74.0,7.0,74.0,63.112329,0,...,0,0,0,0,0,0,0,0,0,0
1992,174.0,26.0,-13.0,7.0,324.0,54.0,4.0,65.0,62.610959,0,...,0,0,0,0,0,0,0,0,0,0


### End of Data Processing Exercise

### Model Training

In [43]:
from sklearn.model_selection import train_test_split

target = ['Popularity']
not_target = [_ for _ in prepped_for_ml.columns if _ not in target]

X = prepped_for_ml.loc[:, not_target]
y = prepped_for_ml[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.30, random_state = 30
)

X_train.describe()

,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Length (Duration),Acousticness,Speechiness,age,Top Genre_acoustic pop,Top Genre_adult standards,...,Top Genre_reggae,Top Genre_reggae fusion,Top Genre_rock-and-roll,Top Genre_scottish singer-songwriter,Top Genre_soft rock,Top Genre_stomp and holler,Top Genre_streektaal,Top Genre_trance,Top Genre_uk pop,Top Genre_yacht rock
count,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,...,1366.000000,1366.000000,1366.000000,1366.000000,1366.000000,1366.0,1366.000000,1366.000000,1366.000000,1366.0
mean,120.068613,60.009924,-8.975135,19.169920,260.962429,29.008047,5.091502,28.649991,0.001464,0.062958,...,0.004392,0.002196,0.000732,0.001464,0.000732,0.0,0.000732,0.000732,0.000732,0.0
std,29.340451,22.253870,3.622797,16.859179,98.510703,29.227000,4.682253,16.359942,0.038250,0.242976,...,0.066154,0.046829,0.027057,0.038250,0.027057,0.0,0.027057,0.027057,0.027057,0.0
min,15.000000,3.000000,-24.000000,2.000000,93.000000,0.000000,2.000000,2.402740,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
25%,99.000000,43.000000,-11.000000,9.000000,210.000000,4.000000,3.000000,14.076712,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
50%,119.000000,62.000000,-8.000000,12.000000,243.000000,18.000000,4.000000,27.249315,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
75%,135.000000,78.000000,-6.000000,23.000000,287.000000,50.000000,5.000000,43.219863,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
max,305.000000,99.000000,-2.000000,99.000000,1412.000000,99.000000,55.000000,65.778082,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,0.0


In [44]:
X_test.describe()

,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Length (Duration),Acousticness,Speechiness,age,Top Genre_acoustic pop,Top Genre_adult standards,...,Top Genre_reggae,Top Genre_reggae fusion,Top Genre_rock-and-roll,Top Genre_scottish singer-songwriter,Top Genre_soft rock,Top Genre_stomp and holler,Top Genre_streektaal,Top Genre_trance,Top Genre_uk pop,Top Genre_yacht rock
count,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,586.000000,...,586.000000,586.000000,586.0,586.0,586.0,586.000000,586.0,586.0,586.0,586.000000
mean,120.443686,58.774744,-9.087031,18.696246,264.870307,28.972696,4.757679,29.386605,0.003413,0.059727,...,0.003413,0.001706,0.0,0.0,0.0,0.001706,0.0,0.0,0.0,0.001706
std,27.054634,21.941892,3.730347,16.353464,83.109547,28.580657,3.610429,16.034605,0.058371,0.237183,...,0.058371,0.041310,0.0,0.0,0.0,0.041310,0.0,0.0,0.0,0.041310
min,18.000000,6.000000,-27.000000,3.000000,125.000000,0.000000,2.000000,2.484932,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
25%,99.250000,41.000000,-11.000000,10.000000,215.250000,4.000000,3.000000,14.745205,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
50%,120.000000,59.000000,-9.000000,12.500000,250.000000,19.000000,4.000000,29.210959,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
75%,136.000000,76.000000,-6.000000,22.000000,290.000000,49.000000,5.000000,42.931507,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
max,204.000000,100.000000,-2.000000,99.000000,859.000000,98.000000,39.000000,64.112329,1.000000,1.000000,...,1.000000,1.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,1.000000


In [45]:
y_train.describe()

,Popularity
count,1366.000000
mean,59.305968
std,14.581226
min,11.000000
25%,49.000000
50%,61.000000
75%,71.000000
max,98.000000


In [46]:
y_test.describe()

,Popularity
count,586.000000
mean,59.873720
std,14.017246
min,17.000000
25%,50.000000
50%,62.000000
75%,71.000000
max,100.000000
